In [ ]:
import warnings
import pandas as pd
import numpy as np

# imports de modulos que estao definidos dentro do projeto
import data
import arima
import timeseries

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
timeseries02_daily = data.load_timeseries02_daily()

timeseries02_daily.plot(figsize=(15, 10), linewidth=2, fontsize=15, style='b.')
plt.grid(True)
plt.xlabel('Ano', fontsize=15)

In [ ]:
timeseries02_monthly = data.load_timeseries02_monthly()

timeseries02_monthly.plot(figsize=(15, 10), linewidth=2, fontsize=15, style='r.')
plt.grid(True)
plt.xlabel('Ano', fontsize=15)

In [ ]:
timeseries.check_stationarity(timeseries02_monthly, window=12)

In [ ]:
timeseries02_monthly_log = np.log(timeseries02_monthly)
timeseries.check_stationarity(timeseries02_monthly_log, window=12)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(timeseries02_monthly)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Select the most recent weeks 
ts_euro_week_log_select = timeseries02_monthly_log[-100:]

plt.subplot(411)
plt.plot(ts_euro_week_log_select.index.to_pydatetime(), ts_euro_week_log_select.values, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(ts_euro_week_log_select.index.to_pydatetime(), trend[-100:].values, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(ts_euro_week_log_select.index.to_pydatetime(), seasonal[-100:].values,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(ts_euro_week_log_select.index.to_pydatetime(), residual[-100:].values, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
ts_euro_week_log_diff = timeseries02_monthly_log - timeseries02_monthly_log.shift()
plt.plot(ts_euro_week_log_diff.index.to_pydatetime(), ts_euro_week_log_diff.values)

In [ ]:
ts_euro_week_log_diff.dropna(inplace=True)
timeseries.check_stationarity(ts_euro_week_log_diff, window=12)

In [ ]:
warnings.filterwarnings("ignore")

arima_params = arima.get_arima_params(
    p_values=range(0, 15),
    d_values=range(0, 10),
    q_values=range(0, 10))
#print(len(arima_params))

#arima.evaluate_models(timeseries02_monthly.values, arima_params)
arima.evaluate_models(ts_euro_week_log_diff.values, arima_params)